# Baseline

In [23]:
from transformers import TrainingArguments
import evaluate
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer
import numpy as np

checkpoint = "distilbert-base-uncased"
#checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [24]:
from datasets import load_dataset

ds = load_dataset("json", data_files=["/data1/malto/shroom/val.model-agnostic.json"])
ds = ds['train'].train_test_split(train_size=0.8)
ds

DatasetDict({
    train: Dataset({
        features: ['ref', 'labels', 'task', 'hyp', 'tgt', 'src', 'model', 'label', 'p(Hallucination)'],
        num_rows: 399
    })
    test: Dataset({
        features: ['ref', 'labels', 'task', 'hyp', 'tgt', 'src', 'model', 'label', 'p(Hallucination)'],
        num_rows: 100
    })
})

In [25]:
ds['train'][100]

{'ref': 'either',
 'labels': ['Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Hallucination',
  'Not Hallucination'],
 'task': 'MT',
 'hyp': "You're not your own lately.",
 'tgt': "You haven't been yourself lately.",
 'src': 'Ты в последнее время сама не своя.',
 'model': '',
 'label': 'Not Hallucination',
 'p(Hallucination)': 0.2}

In [26]:
def preprocess_function(examples):
    inputs = [f"Hyp: < {hyp} > Tgt: < {tgt} >" for hyp, tgt in zip(examples["hyp"], examples['tgt'])]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)
    model_inputs["label"] = [1 if t == "Hallucination" else 0 for t in examples['label']]
    return model_inputs

In [27]:
ds = ds.map(preprocess_function, batched=True)
ds = ds.remove_columns(['hyp', 'ref', 'task', 'p(Hallucination)', 'labels', 'tgt', 'model', 'src'])

In [28]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [29]:
import evaluate
import numpy as np

def compute_metrics(eval_pred):
    accuracy = evaluate.load("accuracy")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [30]:
id2label = {0: "Not Hallucination", 1: "Hallucination"}
label2id = {"Not Hallucination": 0, "Hallucination": 1}

In [31]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
training_args = TrainingArguments(
    output_dir="local_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=7,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=1,
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.677800,0.671031,0.620000
2,0.630500,0.663225,0.550000
3,0.518400,0.585284,0.710000
4,0.325000,0.579898,0.740000
5,0.278200,0.588798,0.750000
6,0.225100,0.605564,0.750000
7,0.186200,0.592125,0.760000


TrainOutput(global_step=175, training_loss=0.47079523588929856, metrics={'train_runtime': 20.6357, 'train_samples_per_second': 135.348, 'train_steps_per_second': 8.48, 'total_flos': 39712535980284.0, 'train_loss': 0.47079523588929856, 'epoch': 7.0})

### Overfitting ?

In [33]:
trainer.evaluate(ds['test'])

{'eval_loss': 0.5798976421356201,
 'eval_accuracy': 0.74,
 'eval_runtime': 1.39,
 'eval_samples_per_second': 71.94,
 'eval_steps_per_second': 5.036,
 'epoch': 7.0}

In [34]:
altro = load_dataset("json", data_files=["/data1/malto/shroom/trial-v1.json"])
#altro = altro['train'].train_test_split(train_size=0.8)
altro = altro.map(preprocess_function, batched=True)
altro = altro.remove_columns(['hyp', 'ref', 'task', 'p(Hallucination)', 'labels', 'tgt', 'model', 'src'])
trainer.evaluate(altro['train'])

{'eval_loss': 0.6088791489601135,
 'eval_accuracy': 0.7,
 'eval_runtime': 1.3085,
 'eval_samples_per_second': 61.137,
 'eval_steps_per_second': 3.821,
 'epoch': 7.0}

In [35]:
altro = load_dataset("json", data_files=["/data1/malto/shroom/val.model-aware.json"])
#altro = altro['train'].train_test_split(train_size=0.8)
altro = altro.map(preprocess_function, batched=True)
altro = altro.remove_columns(['hyp', 'ref', 'task', 'p(Hallucination)', 'labels', 'tgt', 'model', 'src'])
trainer.evaluate(altro['train'])

{'eval_loss': 0.670728862285614,
 'eval_accuracy': 0.6147704590818364,
 'eval_runtime': 1.2924,
 'eval_samples_per_second': 387.649,
 'eval_steps_per_second': 24.76,
 'epoch': 7.0}

In [49]:
altro = load_dataset("json", data_files=["/data1/malto/shroom/val.model-agnostic.json"])
altro['train'][200]

{'ref': 'either',
 'labels': ['Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination',
  'Not Hallucination'],
 'task': 'PG',
 'hyp': "I'll love you forever and ever.",
 'tgt': 'I will love you forever.',
 'src': 'I shall love you always.',
 'model': '',
 'label': 'Not Hallucination',
 'p(Hallucination)': 0.0}

In [48]:
altro = load_dataset("json", data_files=["/data1/malto/shroom/val.model-aware.json"])
altro['train'][200]

{'tgt': '',
 'ref': 'src',
 'label': 'Hallucination',
 'model': 'tuner007/pegasus_paraphrase',
 'labels': ['Hallucination',
  'Hallucination',
  'Hallucination',
  'Not Hallucination',
  'Hallucination'],
 'src': 'All I can do is repeat what I said.',
 'p(Hallucination)': 0.8,
 'task': 'PG',
 'hyp': "I can't do anything else."}

In [37]:
vuoto = load_dataset("json", data_files=["/data1/malto/shroom/train.model-agnostic.json"])
vuoto

DatasetDict({
    train: Dataset({
        features: ['tgt', 'ref', 'model', 'src', 'task', 'hyp'],
        num_rows: 30000
    })
})

In [38]:
vuoto['train'][20000]

{'tgt': '',
 'ref': 'src',
 'model': '',
 'src': "You're not alone, Claire.",
 'task': 'PG',
 'hyp': "You're not alone, claire- -"}